In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip   # download data to notebook


--2024-05-12 13:00:31--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [                <=> ]  41.00M  12.8MB/s    in 12s     

2024-05-12 13:00:43 (3.46 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]



In [2]:
!unzip drugsCom_raw.zip          #unzip data


Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

data=pd.read_csv("drugsComTrain_raw.tsv",sep='\t',engine='python')

data.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [4]:
data.to_csv('drugsComTrain_raw.tsv', index=False, sep='\t')


In [5]:
data = data.drop(["date",'condition','usefulCount','condition','Unnamed: 0']  , axis = 1)

In [6]:
data.columns=['DrugName',"Review",'Rating']
positive_rating = len(data[data['Rating'] >=7.0])
neutral_rating = len(data[(data['Rating'] >=4) & (data['Rating']<7)])
negative_rating = len(data[data['Rating']<=3])
Total_rating = positive_rating+neutral_rating+negative_rating
data.loc[data['Rating'] >=7.0,"ratingSentimentLabel"] ="positive"
data.loc[(data['Rating'] >=4.0) & (data['Rating']<7.0),"ratingSentimentLabel"]= "neutral"
data.loc[data['Rating']<=3.0,"ratingSentimentLabel"] = "negative"
data.head(1000)
data.isnull().sum()


DrugName                0
Review                  0
Rating                  0
ratingSentimentLabel    0
dtype: int64

In [7]:
data = data[data['ratingSentimentLabel'] != 'neutral']
data.head(10)


# convert airline_seentiment to numeric
ratingSentimentLabel = data.ratingSentimentLabel.factorize()
ratingSentimentLabel


(array([0, 0, 0, ..., 1, 0, 0]),
 Index(['positive', 'negative'], dtype='object'))

In [8]:

import numpy as np
from sklearn.impute import SimpleImputer

data=data.dropna()
data=data.drop(['Rating','DrugName'], axis =1 )


In [9]:
data.head()

,Review,ratingSentimentLabel
0,"""It has no side effect, I take it in combinati...",positive
1,"""My son is halfway through his fourth week of ...",positive
3,"""This is my first time using any form of birth...",positive
4,"""Suboxone has completely turned my life around...",positive
5,"""2nd day on 5mg started to work with rock hard...",negative


In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

Review=data.Review.values

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
import re
data['Review']=data['Review'].apply(lambda x:" ".join([re.sub(r"[^\w]"," ", i) for i in x.split()]))
data['Review']=data['Review'].apply(lambda x:" ".join([re.sub(r"['+']", ' ', i) for i in x.split()]))
data['Review']=data['Review'].apply(lambda x:" ".join([re.sub(r'[^\D]', ' ', i) for i in x.split()]))



In [12]:
x=data.Review.values
data['Review']=[i.lower() for i in x ]


data.Review.head(100)

0      it has no side effect i take it in combination...
1      my son is halfway through his fourth week of i...
3      this is my first time using any form of birth ...
4      suboxone has completely turned my life around ...
5       nd day on  mg started to work with rock hard ...
                             ...                        
108    i suffer from chronic pain due to severe arthr...
109    i just started this medication on april  st an...
110    been dealing with restless leg syndrome for ab...
111    i dreaded the side effects and had none i had ...
112    this is a very good medication imo when you ti...
Name: Review, Length: 100, dtype: object

In [13]:
data['Review']=data['Review'].apply(lambda x:" ".join([re.sub(r"['+']", ' ', i) for i in x.split()]))


In [14]:
Review=data.Review.values

def filter_stop_words(train_sentences, stop_words):
    for i, sentence in enumerate(train_sentences):
        new_sent = [word for word in str(sentence).split() if word not in stop_words]
        train_sentences[i] = ' '.join(new_sent)
    return train_sentences

stop_words = set(stopwords.words("english"))
print(stop_words)
train_sentences = filter_stop_words(Review, stop_words)
data['Review']=train_sentences

data['Review'].head(100)





{'or', "haven't", 'what', 'does', 'him', "isn't", 'these', 'theirs', "needn't", 'yourselves', "hadn't", 'doesn', "doesn't", 'do', 'at', 'couldn', 'nor', 'under', 'herself', 'hasn', 'but', "wasn't", 'there', "shouldn't", 'her', 'not', 'itself', 'your', 'again', 'd', 'a', "it's", 're', 'why', "don't", "weren't", 'myself', 'with', 'yourself', 'yours', 'should', 'ma', 'his', 'the', 'further', 'its', 'hadn', 'such', 'has', "you'll", 'shan', 'just', 'am', 'ain', 'been', 'll', 'down', 'out', 'it', "shan't", 'some', 'she', 'to', 'no', 'i', 'where', "you'd", 'about', 'had', "didn't", 'mustn', 'whom', 'this', 'of', 'own', 'haven', 'until', 'me', 'having', 'he', "she's", 'over', 'm', 'how', 'up', 'only', 've', 'an', 'few', 'both', 'between', "that'll", 'for', 'ours', 'in', 'against', 's', 'more', "aren't", 'will', 'because', "mightn't", 'you', 'above', 'after', 'that', 'wasn', 'hers', 'by', 'have', 'ourselves', 'once', "mustn't", 'those', 'is', 'other', 'so', 'doing', 'during', 'too', 'off', 'her

0      side effect take combination bystolic mg fish oil
1      son halfway fourth week intuniv became concern...
3      first time using form birth control glad went ...
4      suboxone completely turned life around feel he...
5      nd day mg started work rock hard erections how...
                             ...                        
108    suffer chronic pain due severe arthritis steno...
109    started medication april st days started seein...
110    dealing restless leg syndrome years kept falli...
111    dreaded side effects none intravenous infusion...
112    good medication imo titrate dose intended noti...
Name: Review, Length: 100, dtype: object

In [15]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
list_data=['a','n','v']
for i in list_data:
   data['Review']=data['Review'].apply(lambda x: " ".join([lemmatizer.lemmatize(word,i) for word in x.split()]))

data['Review'].head(100)

[nltk_data] Downloading package wordnet to /root/nltk_data...


0      side effect take combination bystolic mg fish oil
1      son halfway fourth week intuniv become concern...
3      first time use form birth control glad go patc...
4      suboxone completely turn life around feel heal...
5      nd day mg start work rock hard erection howeve...
                             ...                        
108    suffer chronic pain due severe arthritis steno...
109    start medication april st day start see effect...
110    deal restless leg syndrome year keep fall asle...
111    dread side effect none intravenous infusion th...
112    good medication imo titrate dose intend notice...
Name: Review, Length: 100, dtype: object

In [16]:
from textblob import TextBlob
#data['Review']=data['Review'].apply(lambda x: str(TextBlob(x).correct()))

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
Review=data['Review'].values
tokenizer = Tokenizer(num_words=7000,char_level=False)
tokenizer.fit_on_texts(Review)
print(tokenizer.word_index)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
data.shape

{'take': 1, 'day': 2, 'get': 3, 'go': 4, 'year': 5, 'month': 6, 'work': 7, 'week': 8, 'effect': 9, 'start': 10, 'side': 11, 'feel': 12, 'time': 13, 'pain': 14, 'mg': 15, 'first': 16, 'like': 17, 'good': 18, 'bad': 19, 'use': 20, 'pill': 21, 'period': 22, 'try': 23, 'help': 24, 'make': 25, 'would': 26, 'medication': 27, 'one': 28, 'doctor': 29, 'back': 30, 'life': 31, 'also': 32, 'weight': 33, 'give': 34, 'think': 35, 'medicine': 36, 'since': 37, 'experience': 38, 'anxiety': 39, 'stop': 40, 'sleep': 41, 'still': 42, 'much': 43, 'really': 44, 'hour': 45, 'never': 46, 'control': 47, 'last': 48, 'quot': 49, 'two': 50, 'even': 51, 'say': 52, 'night': 53, 'felt': 54, 'well': 55, 'every': 56, 'great': 57, 'drug': 58, 'gain': 59, 'cramp': 60, 'long': 61, 'thing': 62, 'lose': 63, 'ago': 64, 'prescribe': 65, 'little': 66, 'acne': 67, 'depression': 68, 'birth': 69, 'dose': 70, 'know': 71, 'come': 72, 'put': 73, 'find': 74, 'recommend': 75, 'eat': 76, 'could': 77, 'change': 78, 'problem': 79, 'alm

(141929, 2)

In [17]:
encoded_docs = tokenizer.texts_to_sequences(Review)
print(encoded_docs[0])
padded_sequence = pad_sequences(encoded_docs, maxlen=200)
print(padded_sequence)


[11, 9, 1, 595, 2208, 15, 2437, 1128]
[[   0    0    0 ...   15 2437 1128]
 [   0    0    0 ...   27   91  193]
 [   0    0    0 ...   47  117  237]
 ...
 [   0    0    0 ... 1209  200   27]
 [   0    0    0 ... 1656 1052  127]
 [   0    0    0 ...   66  861  260]]


In [19]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

dataset=pd.read_csv("drugsComTest_raw.tsv",sep='\t',engine='python')

dataset.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4


In [20]:
dataset.to_csv('drugsComTest_raw.tsv', index=False, sep='\t')



In [21]:
dataset.head()


,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4


In [22]:
dataset = dataset.drop(["date",'condition','usefulCount','condition','Unnamed: 0']  , axis = 1)

In [23]:
dataset.columns=['DrugName',"Review",'Rating']
positive_rating = len(dataset[dataset['Rating'] >=7.0])
neutral_rating = len(dataset[(dataset['Rating'] >=4) & (dataset['Rating']<7)])
negative_rating = len(dataset[dataset['Rating']<=3])
Total_rating = positive_rating+neutral_rating+negative_rating
dataset.loc[dataset['Rating'] >=7.0,"ratingSentimentLabel"] ="positive"
dataset.loc[(dataset['Rating'] >=4.0) & (dataset['Rating']<7.0),"ratingSentimentLabel"]= "neutral"
dataset.loc[dataset['Rating']<=3.0,"ratingSentimentLabel"] = "negative"
dataset.head(1000)
dataset.isnull().sum()


DrugName                0
Review                  0
Rating                  0
ratingSentimentLabel    0
dtype: int64

In [24]:
dataset = dataset[dataset['ratingSentimentLabel'] != 'neutral']
dataset.head(10)


# convert airline_seentiment to numeric
ratingSentimentLabel1 = dataset.ratingSentimentLabel.factorize()
ratingSentimentLabel1


(array([   0,    1,    2, ...,   35,  566, 1219]),
 Index(['Mirtazapine', 'Mesalamine', 'Bactrim', 'Contrave', 'Cyclafem 1 / 35',
        'Amitriptyline', 'Methadone', 'Levora', 'Paroxetine', 'Belviq',
        ...
        'Naftin', 'Rozex', 'Blistex', 'Lactase', 'Meprobamate',
        'Empagliflozin / metformin', 'Allergy DN PE', 'Guarana', 'Maprotiline',
        'FluMist'],
       dtype='object', length=2569))

In [25]:

import numpy as np
from sklearn.impute import SimpleImputer

dataset=dataset.dropna()
dataset=dataset.drop(['Rating','DrugName'], axis =1 )
dataset.head()

,Review,ratingSentimentLabel
0,"""I&#039;ve tried a few antidepressants over th...",positive
1,"""My son has Crohn&#039;s disease and has done ...",positive
2,"""Quick reduction of symptoms""",positive
3,"""Contrave combines drugs that were used for al...",positive
4,"""I have been on this birth control for one cyc...",positive


In [26]:
dataset.head(100)

,Review,ratingSentimentLabel
0,"""I&#039;ve tried a few antidepressants over th...",positive
1,"""My son has Crohn&#039;s disease and has done ...",positive
2,"""Quick reduction of symptoms""",positive
3,"""Contrave combines drugs that were used for al...",positive
4,"""I have been on this birth control for one cyc...",positive
...,...,...
109,"""BC from below. Rapid weight gain, swelling an...",negative
111,"""I was getting two hours sleep. Constantly wok...",positive
113,"""AAA ointment. it like... gives you wolverine ...",positive
114,"""I absolutely LOVE this medicine. I lost 30 po...",positive


In [27]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

Review=dataset.Review.values

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
import re
dataset['Review']=dataset['Review'].apply(lambda x:" ".join([re.sub(r"[^\w]"," ", i) for i in x.split()]))
dataset['Review']=dataset['Review'].apply(lambda x:" ".join([re.sub(r"['+']", ' ', i) for i in x.split()]))
dataset['Review']=dataset['Review'].apply(lambda x:" ".join([re.sub(r'[^\D]', ' ', i) for i in x.split()]))
x=dataset.Review.values
dataset['Review']=[i.lower() for i in x ]



import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
#data.Review.head(100)
dataset.Review.tail(5000)

48084    neupro has been a lifesaver in combination wit...
48086    marginally slowed chronic bowel dysfunction bo...
48087    very good i was given samples to try instead o...
48088    taking isotretinoin is the best decision i mad...
48089    my childs grades have suffered more after begi...
48090    most notable change is my appetite i     ve be...
48091    this has been a miracle for me i     ve been o...
48092    this drug is wonderful broke my neck and just ...
48093    easy method reliable increased back pain mood ...
48094    wanted to share my experience i did not take t...
48095    well it has now been about    days since inser...
48096    remember the tune from childhood stranded stra...
48097    i was on depo for six yrs and recently switche...
48098    i have never written a review in my life but u...
48099    took for   years seemed to help with gad after...
48101    i have been using this medicine for   weeks i ...
48102    i took three doses in a row and started gettin.

In [29]:
dataset['Review']=dataset['Review'].apply(lambda x:" ".join([re.sub(r"['+']", ' ', i) for i in x.split()]))
dataset['Review'].head()

0    i ve tried a few antidepressants over the year...
1    my son has crohn s disease and has done very w...
2                          quick reduction of symptoms
3    contrave combines drugs that were used for alc...
4    i have been on this birth control for one cycl...
Name: Review, dtype: object

In [30]:
def filter_stop_words(train_sentences1, stop_words):
    for i, sentence in enumerate(train_sentences1):
        new_sent = [word for word in str(sentence).split() if word not in stop_words]
        train_sentences1[i] = ' '.join(new_sent)
    return train_sentences1

stop_words = set(stopwords.words("english"))
print(stop_words)
train_sentences1 = filter_stop_words(Review, stop_words)
dataset['Review']=train_sentences1

dataset['Review'].head(100)


{'or', "haven't", 'what', 'does', 'him', "isn't", 'these', 'theirs', "needn't", 'yourselves', "hadn't", 'doesn', "doesn't", 'do', 'at', 'couldn', 'nor', 'under', 'herself', 'hasn', 'but', "wasn't", 'there', "shouldn't", 'her', 'not', 'itself', 'your', 'again', 'd', 'a', "it's", 're', 'why', "don't", "weren't", 'myself', 'with', 'yourself', 'yours', 'should', 'ma', 'his', 'the', 'further', 'its', 'hadn', 'such', 'has', "you'll", 'shan', 'just', 'am', 'ain', 'been', 'll', 'down', 'out', 'it', "shan't", 'some', 'she', 'to', 'no', 'i', 'where', "you'd", 'about', 'had', "didn't", 'mustn', 'whom', 'this', 'of', 'own', 'haven', 'until', 'me', 'having', 'he', "she's", 'over', 'm', 'how', 'up', 'only', 've', 'an', 'few', 'both', 'between', "that'll", 'for', 'ours', 'in', 'against', 's', 'more', "aren't", 'will', 'because', "mightn't", 'you', 'above', 'after', 'that', 'wasn', 'hers', 'by', 'have', 'ourselves', 'once', "mustn't", 'those', 'is', 'other', 'so', 'doing', 'during', 'too', 'off', 'her

0      "I&#039;ve tried antidepressants years (citalo...
1      "My son Crohn&#039;s disease done well Asacol....
2                             "Quick reduction symptoms"
3      "Contrave combines drugs used alcohol, smoking...
4      "I birth control one cycle. After reading revi...
7      "This great me. I&#039;ve 2 weeks last week I ...
8      "Ive Methadone ten years currently,I trying ge...
9      "I pill almost two years. It work far getting ...
10     "Holy Hell exactly I feel. I taking Brisdelle ...
12     "This waste money. Did curb appetite make feel...
13                             "No problems, watch eat."
14     "Ditto rebound sleepless discontinued. I done ...
15     "A doctor ER prescribed 200 mg Provigil I firs...
16     "I smoked 50+ years. Took one week it. I didn&...
17     "So I Ginanvi 3 months I switched pill due hig...
19       "After 1 dose ciprofloxacn, I felt 99% better."
20     "If I could give 0, I would absolutely so. Sta...
21     "I happy samples provide

In [31]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
list_data=['a','v','n']
for i in list_data:
   dataset['Review']=dataset['Review'].apply(lambda x: " ".join([lemmatizer.lemmatize(word,i) for word in  x.split()]))



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
from textblob import TextBlob
#data['Review']=data['Review'].apply(lambda x: str(TextBlob(x).correct()))

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer1 = Tokenizer(num_words=7000,char_level=False)
tokenizer1.fit_on_texts(Review)
print(tokenizer1.word_index)
vocab_size1 = len(tokenizer1.word_index) + 1
print(vocab_size1)
dataset.shape

{'i': 1, '039': 2, 'it': 3, 't': 4, 'day': 5, 'm': 6, 'side': 7, 'taking': 8, 'years': 9, 've': 10, 's': 11, 'effects': 12, 'pain': 13, 'take': 14, 'first': 15, 'my': 16, '2': 17, 'the': 18, 'like': 19, 'started': 20, 'months': 21, 'get': 22, 'days': 23, 'time': 24, '3': 25, 'feel': 26, 'would': 27, 'this': 28, 'back': 29, 'one': 30, 'pill': 31, 'weeks': 32, 'doctor': 33, 'medication': 34, 'took': 35, 'week': 36, 'life': 37, 'me': 38, 'weight': 39, 'got': 40, 'also': 41, 'period': 42, 'month': 43, 'since': 44, 'don': 45, 'anxiety': 46, 'bad': 47, '5': 48, 'much': 49, 'still': 50, 'really': 51, 'never': 52, 'medicine': 53, 'work': 54, '4': 55, 'better': 56, 'quot': 57, 'went': 58, 'now': 59, 'two': 60, 'felt': 61, 'every': 62, 'well': 63, 'great': 64, 'control': 65, 'even': 66, 'night': 67, 'good': 68, 'year': 69, 'didn': 70, '1': 71, 'go': 72, 'sleep': 73, 'tried': 74, 'works': 75, 'ago': 76, '6': 77, 'after': 78, 'drug': 79, '10': 80, 'little': 81, 'made': 82, 'depression': 83, 'mg': 

(47278, 2)

In [33]:
encoded_docs = tokenizer1.texts_to_sequences(Review)
print(encoded_docs[1])
padded = pad_sequences(encoded_docs, maxlen=200)
print(padded)


[16, 580, 2308, 2, 11, 794, 295, 63, 2848, 298, 1308, 3307, 7, 12, 298, 146, 154, 2553, 691, 342, 5, 30, 24, 1, 2, 10, 180, 267, 2171, 1866, 368, 2525]
[[   0    0    0 ... 2170  512   38]
 [   0    0    0 ... 1866  368 2525]
 [   0    0    0 ...  896 1824  114]
 ...
 [   0    0    0 ...  273  327  180]
 [   0    0    0 ...   13  371  214]
 [   0    0    0 ...    0    3   75]]


In [34]:
# Build the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

embedding_vector_length = 50
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           1729650   
                                                                 
 spatial_dropout1d (Spatial  (None, 200, 50)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 50)                20200     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1024)              52224     
                                                                 
 dense_1 (Dense)             (None, 1)                 1025      
                                                        

In [35]:

history = model.fit(padded_sequence,ratingSentimentLabel[0],validation_split=0.1,epochs=25, batch_size=800)


Epoch 1/25
160/160 [==============================] - 133s 786ms/step - loss: 0.4199 - accuracy: 0.8148 - val_loss: 0.3255 - val_accuracy: 0.8647
Epoch 2/25
160/160 [==============================] - 113s 704ms/step - loss: 0.3159 - accuracy: 0.8671 - val_loss: 0.3148 - val_accuracy: 0.8693
Epoch 3/25
160/160 [==============================] - 111s 696ms/step - loss: 0.2991 - accuracy: 0.8756 - val_loss: 0.3075 - val_accuracy: 0.8739
Epoch 4/25
160/160 [==============================] - 110s 689ms/step - loss: 0.2869 - accuracy: 0.8810 - val_loss: 0.3086 - val_accuracy: 0.8761
Epoch 5/25
160/160 [==============================] - 109s 680ms/step - loss: 0.2757 - accuracy: 0.8857 - val_loss: 0.3055 - val_accuracy: 0.8742
Epoch 6/25
160/160 [==============================] - 105s 660ms/step - loss: 0.2693 - accuracy: 0.8890 - val_loss: 0.2971 - val_accuracy: 0.8766
Epoch 7/25
160/160 [==============================] - 108s 677ms/step - loss: 0.2608 - accuracy: 0.8927 - val_loss: 0.2960 -

**Predict Data**

In [36]:
#test_word='Smoking it was a constant companion for so long and I honestly thought I die of COPD on oxygen, begging for that last smoke. Chantix truly changed all of that. It completely eliminated the physical craving, leaving the rest up to me.'
test_word='This is the first birth control pill Ive ever been on and have no major complaints.  Im a healthy height/weight and exercise regularly.  I just finished my third pack.  During the first month, I had breakthrough bleeding (as expected with most pills) off and on, but that was it.  My breasts are a little more tender, but its nothing bothersome.  The pill definitely hasnt bothered my sex drive either.'
tw = tokenizer1.texts_to_sequences([test_word])
tw
pad = pad_sequences(tw,maxlen=200)
pad


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   28,
         593,   18,   15,   87,   65,   31, 1537,  108,  471,  238,  174,
         320,   96,  375, 1308,  784, 

In [37]:
prediction = int(model.predict(pad).round().item())
print(ratingSentimentLabel1[1][prediction])


1/1 [==============================] - 0s 376ms/step
positive


In [38]:
test_word='Ive tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia anxiety'
tw = tokenizer1.texts_to_sequences([test_word])
tw
pad = pad_sequences(tw,maxlen=200)
pad


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [39]:
prediction = int(model.predict(pad).round().item())
print(ratingSentimentLabel1[1][prediction])

1/1 [==============================] - 0s 125ms/step
negative


In [40]:
test_word='My son has Crohns disease and has done very well on the Asacol.He has no complaints and shows no side effects '
tw = tokenizer1.texts_to_sequences([test_word])
tw
pad = pad_sequences(tw,maxlen=200)
pad


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [41]:
prediction = int(model.predict(pad).round().item())
print(ratingSentimentLabel1[1][prediction])

1/1 [==============================] - 0s 101ms/step
positive


In [42]:
test_word='My Dr had put me on the Losartan. Right after I started taking it, my whole back was in so much pain. I had told my Dr about the pain and I believed it was the Losartan. He said I had a physically demanding job. Keep taking it. After being in so much pain, I couldnt work I emailed him and explained my pain. He said there has been no case studies of it causing back pain to keep taking it. After telling him three times I was in pain due to this medicine and he still would not help me I am switching Drs'
tw = tokenizer1.texts_to_sequences([test_word])
tw
pad = pad_sequences(tw,maxlen=200)
pad

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   16,  231,
         299,   99,   38,  238,   18, 3806,  167,   78,    1,   20,    8,
           3,   16,  363,   29,  331,  164,   86,   49,   13,    1,  299,
         186,   16,  231,  522,   18,   13,  174,    1, 3519,    3,  331,
          18, 3806,  298,  200,    1,  299,  248, 1743,  422,  265,    8,
           3,   78, 1594,  164,   86, 

In [43]:
prediction = int(model.predict(pad).round().item())
print(ratingSentimentLabel1[1][prediction])

1/1 [==============================] - 0s 101ms/step
positive


In [44]:
test_word="LEAST effective I have ever come across.  At its low price point, feel free to give it a try and maybe you will be luckier?  Everyones sleep conditions are different.  But if you get hyper after benadryl, expect the same reaction to this drug."
tw = tokenizer1.texts_to_sequences([test_word])
tw
pad = pad_sequences(tw,maxlen=200)
pad

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [45]:
prediction = int(model.predict(pad).round().item())
print(ratingSentimentLabel1[1][prediction])

1/1 [==============================] - 0s 96ms/step
positive
